# 데이터사이언스 (0010085001)

## Exercise 15: PyTorch (Linear Models)

* source from : https://github.com/L1aoXingyu/pytorch-beginner

### 1. Linear Regression

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable

In [ ]:
# 데이터 생성
x_train = np.array([[3.3], [4.4], [5.5], [6.71], [6.93], [4.168],
                    [9.779], [6.182], [7.59], [2.167], [7.042],
                    [10.791], [5.313], [7.997], [3.1]], dtype=np.float32)

y_train = np.array([[1.7], [2.76], [2.09], [3.19], [1.694], [1.573],
                    [3.366], [2.596], [2.53], [1.221], [2.827],
                    [3.465], [1.65], [2.904], [1.3]], dtype=np.float32)

# numpy.array -> torch.tensor
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

In [ ]:
# Linear regression 모델 생성

### torch.nn.Module
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html#module

class linearRegression(nn.Module):
    def __init__(self):
        super(linearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)  # input and output is 1 dimension

    def forward(self, x):
        out = self.linear(x)
        return out

model = linearRegression()

In [ ]:
# Loss (cost) function 설정

### torch.nn.MSELoss
# https://pytorch.org/docs/stable/nn.html#loss-functions

criterion = nn.MSELoss()

In [ ]:
# Optimizer 설정

### torch.optim.SGD (Stochastic gradient descent)
# https://pytorch.org/docs/stable/optim.html

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [ ]:
# 모델 학습

num_epochs = 1000

for epoch in range(num_epochs):
    inputs = x_train
    target = y_train

    # forward
    out = model(inputs)
    loss = criterion(out, target)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f'Epoch[{epoch+1}/{num_epochs}], loss: {loss.item():.6f}')

### torch.nn.Module.eval()
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.eval
model.eval()

### torch.no_grad()
# https://pytorch.org/docs/stable/generated/torch.no_grad.html#torch.no_grad
with torch.no_grad():
    predict = model(x_train)

predict = predict.data.numpy()

In [ ]:
# 학습 결과 시각화

fig = plt.figure(figsize=(10, 5))
plt.plot(x_train.numpy(), y_train.numpy(), 'ro', label='Original data')
plt.plot(x_train.numpy(), predict, label='Fitting Line')

plt.legend() 
plt.show()

In [ ]:
# 모델 저장 (즉, 학습으로 얻은 weights 를 저장)

torch.save(model.state_dict(), './linear.pth')

### 2. Logistic Regression

In [ ]:
import time

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [ ]:
# 학습 파라미터 설정

batch_size = 64
learning_rate = 1e-3
num_epochs = 10

In [ ]:
# 데이터셋 설정

### torchvision.datasets.FashionMNIST
# https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html#torchvision.datasets.FashionMNIST

train_dataset = datasets.FashionMNIST(root='./data', 
                                      train=True, download=True,
                                      transform=transforms.ToTensor())

test_dataset = datasets.FashionMNIST(root='./data', train=False, 
                                     transform=transforms.ToTensor())

In [ ]:
# 데이터 로더 설정

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Logistic regression 모델 생성

class Logistic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logistic_Regression, self).__init__()
        self.logistic = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.logistic(x)
        return out

model = Logistic_Regression(28 * 28, 10)

use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()

In [ ]:
# Loss (cost) function 설정

criterion = nn.CrossEntropyLoss()

In [ ]:
# Optimizer 설정

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# 모델 학습

for epoch in range(num_epochs):
    print('*' * 10)
    print(f'epoch {epoch+1}')
    
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    
    model.train()
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)  # 将图片展开成 28x28
    
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
    
        # forward
        out = model(img)
    
        # calculate error
        loss = criterion(out, label)
    
        running_loss += loss.item()


        _, pred = torch.max(out, 1)
        running_acc += (pred==label).float().mean()
    
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')

    # 모델 평가 (test)    
    model.eval()

    eval_loss = 0.
    eval_acc = 0.

    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)

        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        
        with torch.no_grad():
            out = model(img)
            loss = criterion(out, label)
        
        eval_loss += loss.item()
        _, pred = torch.max(out, 1)
        eval_acc += (pred == label).float().mean()

    print(f'Test Loss: {eval_loss/len(test_loader):.6f}, Acc: {eval_acc/len(test_loader):.6f}')
    print(f'Time:{(time.time()-since):.1f} s')

In [ ]:
# 모델 저장 (즉, 학습으로 얻은 weights 를 저장)

torch.save(model.state_dict(), './logstic.pth')